In [1]:
import numpy as np
import tensorflow as tf 
import sys
import os
import glob
import dotenv
import pandas as pd
from dotenv import load_dotenv, find_dotenv
sys.path.append(os.getenv('src_dir'))
processed_data_dir = os.getenv('processed_data_dir')

In [2]:

from src.models.model_arcitectures._model_quantile_simp import quantile_model
from src.models.model_arcitectures._model_quantule_v2 import quantile_model as qmodel_v2
from src.models.model_arcitectures._fcn_quantile import fcn_quantile
from src.models.model import Model
from src.models._util import seed_everything

seed_everything(42)

In [3]:
dataset = np.load(glob.glob(processed_data_dir+'/*')[1],allow_pickle=True)

In [4]:
dataframe = pd.DataFrame(dataset['dataframe'], columns = ['image_id','Sar_Incidence','sar_type','mmsi','cog','sog','shape'])

In [5]:
#X_0 = dataset["imgs_training_clipped_scaled"]
#X_90 = np.rot90(dataset["imgs_training_clipped_scaled"],k=1,axes=(1,2))
#X_180 = np.rot90(dataset["imgs_training_clipped_scaled"],k=2,axes=(1,2))
#X_270 = np.rot90(dataset["imgs_training_clipped_scaled"],k=3,axes=(1,2))

#X = np.concatenate([X_0, X_90, X_180,X_270])
#X_targets = np.concatenate([dataset["targets_training_scaled"], dataset["targets_training_scaled"], dataset["targets_training_scaled"],dataset["targets_training_scaled"]])
#X_metadata = np.concatenate([dataset["metadata_training_scaled"], dataset["metadata_training_scaled"], dataset["metadata_training_scaled"],dataset["metadata_training_scaled"]])


In [13]:
X = np.array([im.astype(float) for im in dataset["imgs_training_clipped_scaled"]])
Meta = np.array([met.astype(float) for met in dataset["metadata_training_scaled"]])
y = np.array([met.astype(float) for met in dataset["targets_training_scaled"]])

/var/folders/82/fwjm77x535gfxv2dlbyp5l400000gp/T/ipykernel_95832/96545764.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([im.astype(float) for im in dataset["imgs_training_clipped_scaled"]])


In [14]:
X = tf.ragged.constant(X).to_tensor()
Meta = tf.ragged.constant(Meta).to_tensor()
y = [tf.ragged.constant(y[:,0]).to_tensor(),tf.ragged.constant(y[:,1]).to_tensor(),tf.ragged.constant(y[:,2]).to_tensor()]

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'to_tensor'

In [ ]:
with Model() as M:
    M.data_load(X,Meta,y)
    M.model_load(which_model='fcn_quantile')
    

Virtual devices cannot be modified after being initialized
quantile
(None, None, None, 2)


In [ ]:
M.model.model.summary()

Model: "quantile_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs_img (InputLayer)        [(None, None, None,  0           []                               
                                 2)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   13056       ['inputs_img[0][0]']             
                                256)                                                              
                                                                                                  
 tf.nn.elu (TFOpLambda)         (None, None, None,   0           ['conv2d[0][0]']                 
                                256)                                                    

In [ ]:
M.model_train(epochs=10,verbose=0)

TypeError: in user code:

    File "/opt/homebrew/Caskroom/miniforge/base/envs/env_tfp/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/env_tfp/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniforge/base/envs/env_tfp/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/env_tfp/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/opt/homebrew/Caskroom/miniforge/base/envs/env_tfp/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from 